---
title: "Soil Water Content and Ecosystem Respiration analysis"
format:
  html:
    code-fold: true
---


## Introduction

This is an analysis of Soil Water Content (SWC) and and Ecosystem Respiration (RECO) data.

GIVE A BIT OF BACKGROUND ON THE DATASET IE LEVELS ETC; JUSTIFY WHY ONLY 4 LEVELS

First, lets import the modules we will need to complete the analysis:


In [ ]:
import os

# data access and manipulation modules
import pandas as pd 
import numpy as np
import csv

# plotting libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go # interactive plots
import matplotlib.dates as mdates
import datetime
from datetime import datetime
from matplotlib.backends.backend_pdf import PdfPages # saves multiple individual plots in 1 doc in case of not faceted plot
from collections import defaultdict # for plotting layers of each station

Now, lets access the data we will be analysing.


In [ ]:
reco_data = './data/reco/'
soil_water_data = './data/swc/'
filtered_station_locations = './data/filtered-fluxnet-station-coords.csv'

I wrote this code to also work with the many other variables and indices available from FLUXNET, so I wrote a lot of functions that can be used for many of them (rather than having to write a fresh code chunk each time). This is what now follows. Explanations of the function follows before each code block. 

The function below takes the datasets to be analysed, here SWC and RECO, and puts them into a more flexible form for te purposes of analysis and visualisation - a dictionary. The data directly from fluxnet is in the form of individual .txt files for each station. Here, I take these individual files and all their contents, and stitch them together into a dictionary, to be accessed by the station name as a key. 


In [ ]:
def process_data_files(directory):
    data_dict = {}
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        station_name = os.path.splitext(os.path.basename(file_path))[0].replace('_daily_avg', '')

        time = []
        data = []

        with open(file_path, 'r') as datafile:
            plotting = csv.reader(datafile, delimiter='\t')
            next(plotting)  # skips header row
            for ROWS in plotting:
                time.append(datetime.strptime(ROWS[0], '%Y-%m-%d'))
                data.append(float(ROWS[1]))

        data_dict[station_name] = {'time': time, 'data': data}

    return data_dict
  ```


For some variables, it is interesting to take the log of its value; this is what this function does, and it creates a new directionary with the same name + log i.e. log_reco_data_dict.

 ```{python}
  def log_transfo_data_dict(original_data_dict):
    log_transformed_dict = {}

    for station_name, data_info in original_data_dict.items():
        log_transformed_data = np.log(data_info['data'])
        log_transformed_dict[station_name] = {'time': data_info['time'], 'data': log_transformed_data}

    return log_transformed_dict
  ```


  The function below takes whichever variables you want - again, here only SWC and RECO - and calculates various regression metrics of them, namely the r sqaured metric, the Pearson correlation coefficient, as well as the linear regression metrics (slope and intercept). 


 ```{python}
def calculate_regression_metrics(x, y):
    x = np.array(x)
    y = np.array(y)

    # find indices where either x or y is NaN
    nan_indices = np.isnan(x) | np.isnan(y)

    # exclude NaN values from both x and y
    x_valid = x[~nan_indices]
    y_valid = y[~nan_indices]

    if len(x_valid) == 0 or len(y_valid) == 0:
        # if no valid data points are available, return NaN for all metrics
        return np.nan, np.nan, np.nan, np.nan

    correlation_coefficient = np.corrcoef(x_valid, y_valid)[0, 1]

    # linear regression for slope and intercept
    slope, intercept = np.polyfit(x_valid, y_valid, 1)

    residuals = y_valid - (slope * x_valid + intercept)
    ss_residual = np.sum(residuals**2)
    ss_total = np.sum((y_valid - np.mean(y_valid))**2)
    r_squared = 1 - (ss_residual / ss_total)

    return correlation_coefficient, r_squared, slope, intercept

Once the metrics are calculated,  it is necessary to ave them stored somewhere - so here is a function to store them in a dictionary. It extracts the prefix of the station for the first data dictioanry, which indicates its name, and compares it with the prefix from the second data dictionary. If the names match and the data is not empty at that station, then it moves on to the next step - else it throws an error message to alert you of the emoty data. The next step is to ensure we are only calculating the metrics over common time steps,; the data is thus cropped to the later beginning and earlier end of the 2 datasets. The metrics are then stored with astring as the key and the value as the value, and the whole thing is stored in the dictionary that was intialised at the beginning of the function.


In [ ]:
def calculate_and_store_all_levels(dict1, dict2): 
    all_levels_metrics = {} # initialise emoty dictionary

    for key1, values1 in dict1.items():
      # the next 3 lines are to get the station name prefix out of the first dictionary
        split_parts = key1.split('_')
        station_name = split_parts[0]
        level_name = split_parts[2]
        matching_keys = [key2 for key2 in dict2.keys() if key2.startswith(station_name)]

        if matching_keys:
            for key2, values2 in dict2.items():
                if key2.startswith(station_name) and 'data' in values2:
                    x = np.array(values1.get('data', []))
                    y = np.array(values2.get('data', []))

                    if not (np.iterable(x) and np.iterable(y)):
                        print(f"Invalid data format for {key1} or {key2}")
                        continue

                    if len(x) == 0 or len(y) == 0:
                        print(f"Empty data array for {key1} or {key2}")
                        continue

                    min_len = min(len(x), len(y))
                    x = x[:min_len]
                    y = y[:min_len]

                    correlation_coefficient, r_squared, slope, intercept = calculate_regression_metrics(x, y)

                    level_metrics = {
                        'correlation_coefficient': correlation_coefficient,
                        'r_squared': r_squared,
                        'slope': slope,
                        'intercept': intercept
                    }

                    station_and_level = f"{station_name}_{level_name}"

                    if station_and_level not in all_levels_metrics:
                        all_levels_metrics[station_and_level] = {}

                    all_levels_metrics[station_and_level][station_name] = level_metrics

    return all_levels_metrics

I want to keep all my functions in one place, so I wont yet laod the results of the metric calculation; it will be found below, after the function definitions. 

If you want to save the results of all this work so far externally to a csv file, this is te function for you ! However, we wont do that here, since this is meant to be a standalone reproducible document. Here is the code chunk anyway: 


In [ ]:
echo: true
eval: false
def metrics_to_csv(metrics_dict, name1, name2):

    path = '/path/to/where/you/want/to/save/the/csv/'
    file_name = f"{name1}_{name2}_something_else.csv"
    file_path = path+file_name
    with open(file_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)

        header = ['Station', 'Correlation Coefficient', 'R-squared', 'Slope', 'Intercept']
        writer.writerow(header)


        for station in metrics_dict.keys():
            inner_dict = metrics_dict[station][next(iter(metrics_dict[station]))]
            row = [station, inner_dict['correlation_coefficient'], inner_dict['r_squared'], inner_dict['slope'], inner_dict['intercept']]
            writer.writerow(row)

    print(f'Successfully saved to {file_path}')

As briefly mentioned above, the levels feature of the FLUXNET data is very interesting. So, we ave some functions to filter for that. First we need to check that the levels each have the same time step - it could be, that at a particular time step, levels 1,2, and 4 were recorded and not level 3, due to equipment failure for example. In this case, we do not want it, because we want a consistent time series. So we check first for this time step consistency:


In [ ]:
def remove_missing_time_steps(dict1):
    for station_name, levels_data in gst_filtered.items():
        station_prefix = station_name.split('_')[0]

        level_keys = [key for key in dict1 if key.startswith(station_prefix)]

        if len(level_keys) < 2:
            continue

        # create a dictionary to store time data for each level
        time_data = {level: dict1[level]['time'] for level in level_keys}

        reference_level = level_keys[0]  # first level as the reference
        for current_level in level_keys[1:]:
            # Update time for the current level
            i = 0  # initialize index
            while i < len(time_data[current_level]):
                timestep = time_data[current_level][i]

                if timestep not in time_data[reference_level]:
                    print(f'Removing timestep {timestep} from {current_level}')

                    # Remove corresponding 'data' value
                    del dict1[current_level]['data'][i]

                    # Remove the timestep from the list
                    del time_data[current_level][i]
                else:
                    i += 1  # move to the next timestep

            # Update dictionary with the modified time data
            dict1[current_level]['time'] = time_data[current_level]

    return dict1

Something I want to look at is some patterns between levels and stations, which involes retaining time steps at each station that have data for 4 levels only - not 3, or less. That is what this function is for - to refine the dictionaries to only include the data if it exists in fulll i.e. at all levels. 


In [ ]:
def four_levels_filter(data_dict, data_name):
    desired_levels_stations = {}

    for station_name in set(station.split('_')[0] for station in data_dict.keys()):
        # initialize a dictionary for the station
        station_data = {}

        for station in data_dict:
            if station.startswith(station_name):
                # extract level from the station name
                level = station.split('_')[-1][-2:]

                # get data and time_steps from the dictionary
                data = data_dict[station].get('data', [])
                time_steps = data_dict[station].get('time', [])

                # store data and time_steps in the station_data dictionary
                station_data[level] = {'data': data, 'time': time_steps}

        # check if the station has exactly 4 levels before storing in the dictionary
        if len(station_data) == 4:
            # Find common time steps among all levels
            common_time_steps = set.intersection(*[set(station_data[level]['time']) for level in station_data])

            # Filter data and time_steps to keep only common time steps
            for level in station_data:
                # Filter data and time_steps to keep only common time steps
                station_data[level]['data'] = [value for value, ts in zip(station_data[level]['data'], station_data[level]['time']) if ts in common_time_steps]
                station_data[level]['time'] = list(common_time_steps)

            desired_levels_stations[station_name] = station_data

            
 # Create a dictionary with the desired structure
    four_levels_only_gst_dict = {}
    
    for station_name, levels_data in desired_levels_stations.items():
        for level, data in levels_data.items():
            new_key = f"{station_name}_{data_name}_{level}"
            four_levels_only_gst_dict[new_key] = {'time': data['time'], 'data': data['data']}
    
    return four_levels_only_gst_dict

This function simply ensures that whichever 2 variables you want to play with cover the same period of time - if one i slonger than the other, the longer bit is removed by this function.

In [ ]:
def filter_level_data_common_time(dict1, dict2):    # order is important! dct1 is level data ie swc and dict2 is non leveled data ie reco
    dict1_filtered = {}
    dict2_filtered = {}

    for station_name, data1 in dict2.items():
        station_prefix = station_name.split('_')[0]
      #  print(station_name)
        for station_level_name, data2 in dict1.items():
            if station_level_name.startswith(station_prefix):
                level_name = station_level_name.split('_')[2] # the _l1_ etc bit of the name
                # print(level_name)
                #common_time_range = set(data1.get('time', [])).intersection(set(data2.get('time', [])))
                common_time_range = [time for time in data1.get('time', []) if time in data2.get('time', [])]

               # print(common_time_range)
            
                if common_time_range:

                    dict1_key = f"{station_level_name}"
                    dict1_filtered[dict1_key] = {'time': list(common_time_range), 'data': []}

                    dict1_filtered[dict1_key]['data'] = [value for time, value in zip(data2['time'], data2['data']) if time in common_time_range]
                    #print(len(dict1_filtered[dict1_key]['data']))

                    dict2_key = f"{station_name}_{level_name}"  # make new name for reco with level in it
                    dict2_filtered[dict2_key] = {'time': list(common_time_range), 'data': []}

                    dict2_filtered[dict2_key]['data'] = [value for time, value in zip(data1['time'], data1['data']) if time in common_time_range]
#                    print(len(dict2_filtered[dict2_key]['data']))

    return dict1_filtered, dict2_filtered

In [ ]:
def box_plot_metrics_over_levels(metric_dict, metric_name, xlabel, ylabel):
    # convert metric_dict to a pandas DataFrame for easier plotting
    data = {'Level': [], 'Metric': [], 'Value': []}

    for station_metric, station_metrics in metric_dict.items():
        level_name = station_metric.split('_')[1]
        correlation_coefficient = station_metrics[station_metric.split('_')[0]][metric_name] # bad naming here, but its for the metric inthe metrics to plot thing!

        data['Level'].append(level_name)
        data['Metric'].append(metric_name)
        data['Value'].append(correlation_coefficient)

    df = pd.DataFrame(data)

    fig, ax = plt.subplots(figsize=(10, 6))
    levels = sorted(df['Level'].unique())  # Ensure levels are sorted
    box_data = [df[df['Level'] == level]['Value'].tolist() for level in levels]
    bp = ax.boxplot(box_data, labels=levels, showfliers=True, patch_artist=True)

    # fill boxes with the same colour as levels from scatter
    colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red']
    for box, color in zip(bp['boxes'], colors):
        box.set(facecolor=color, alpha=0.7)
    
    ax.set_title(f'Box Plot of {metric_name}, stations with 4 levels only, common time steps between levels - {xlabel} and {ylabel}')
    ax.set_xlabel('Level')
    ax.set_ylabel(metric_name)
    
    return fig

So, that was a lot, but those functions make up most of the code! Now we can move on to some analysis. Lets use the 'process_data_files' function to stitch together our datasets, and also make a lof version of the RECO data set too, since that could be interesting:


In [ ]:
reco_data_dict = process_data_files(reco_data)
log_reco_data_dict = log_transfo_data_dict(reco_data_dict)
swc_data_dict = process_data_files(soil_water_data)

First, we remove any inconsistencies in the levels data:


In [ ]:
# for retaining time steps with 4 levels only. so obvi do only with data that has levels (i.e. not reco!) 

remove_missing_time_steps(swc_data_dict) # if you want, you can name this and make it a seperate dictionary, to have one with 4 levels only, and one without the levels filtering. for te future of tis project, i am interested in the 4 levels only, so i will stick with that. 

Then we filter for the stations have have exclusively 4 levels over the whole time series.


In [ ]:
four_levels_swc = four_levels_filter(swc_data_dict, 'swc')

Then, we simply crop the 2 data sets we want to compare - here SWC and RECO, and SWC and logRECO, such that they have the same period.


In [ ]:
swc_filtered, reco_swc_filtered = filter_level_data_common_time(four_levels_swc, reco_data_dict)
swc_logfiltered, logreco_swc_filtered = filter_level_data_common_time(four_levels_swc, log_reco_data_dict)

Now, we calculate the regression metrics for this data.


In [ ]:
swc_reco_four_levels_metrics = calculate_and_store_all_levels(swc_filtered, reco_swc_filtered)
swc_four_levels_logreco_metrics = calculate_and_store_all_levels(swc_logfiltered, logreco_swc_filtered)

ADD BOX PLOT PLOTS IN HERE

If you wanted to save these boxplots as a pdf, use this function: 


In [ ]:
# just copy and paste! simply remember to change the name in the file path, and the variables specificied in the figure:

echo: true
eval: false

metrics_to_plot = ['correlation_coefficient', 'r_squared', 'slope', 'intercept'] # to automate function call

with PdfPages('C:/Users/violi/Documents/ESDSRS/STD/soil-moisture-ghc/results-test/4_levels_same_timesteps_swc_reco_box_plots.pdf') as pdf_pages:
    for metric in metrics_to_plot:
        fig = box_plot_metrics_over_levels(swc_reco_four_levels_metrics, metric, 'SWC', 'RECO')
        pdf_pages.savefig(fig, bbox_inches='tight')
        plt.close(fig)

And if you wanted to save these metrics as a csv file, use this code chunk:


In [ ]:
echo: true
eval: false

metrics_to_csv(swc_reco_four_levels_metrics, 'SWC', 'RECO')
metrics_to_csv(swc_four_levels_logreco_metrics, 'SWC', 'logRECO')

So far, so cool! But its lacking a bit of explicit spatio temporal energy. So, lets firstly plot the stations on a world map so we can see where we are looking at. 


This is how I filtered the full station coordinate data set for only the ones that were present in my final datasets. You dont need to run this in the final reproducible workflow - but include but Ive included it as a snippet for full reproducibility.


In [ ]:
echo: true
eval: False
df_coords = pd.read_csv(station_location_csv)
station_names_from_dict = {key.split('_')[0] for key in swc_data_dict.keys()}
df_filtered = df_coords[df_coords['station'].isin(station_names_from_dict)]

# save the filtered DataFrame to a new CSV file
output_csv_file_path = '/pat/to/save/filtered/coord/data/
df_filtered.to_csv(output_csv_file_path, index=False)

print(f"Filtered CSV file has been saved to {output_csv_file_path}")

Using this freshly filtered dataset, lets plot the locations on a really nice, interactive plot. You can zoom in and out here, and hover over a point to see the name!


In [ ]:
# this is an interactive plot!! you can scroll and click!! and it tells you the name and coords of the station!!

# load the location file
df = pd.read_csv(filtered_station_locations)

# ensure latitude and longitude are numeric
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['lon'] = pd.to_numeric(df['lon'], errors='coerce')

# create the map
fig = go.Figure(data=go.Scattergeo(
    lon = df['lon'],
    lat = df['lat'],
    text = df['station'],  
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'red',
        line = dict(
            width = 1,
            color = 'rgba(68, 68, 68, 0)'
        )
    )
))

# layout for a world map with just outlines
fig.update_layout(
    title = dict(
        text = 'FLUXNET Station Locations',
        x = 0.5,  # this centres the title
        y = 0.95  # and this moves it down a little so its not touching the top
    ),
    geo = dict(
        showland = True,
        showcoastlines = True,
        showcountries = True,
        countrycolor = 'rgb(204, 204, 204)',
        coastlinecolor = 'rgb(204, 204, 204)',
        landcolor = 'rgb(255, 255, 255)',
        projection_type = 'natural earth',  # changed to natural earth projection - can also do orthographic, mercator, equirectangular...etc etc
        showocean = True,
        oceancolor = 'rgb(255, 255, 255)',
        showframe = False,
        lonaxis = dict(
            range = [-180, 180],  # crop longitude here if you dont want full map
        ),
        lataxis = dict(
            range = [-90, 90],  # same with latitude
        ),
    ),
    width = 1000,  # width of the figure
    height = 600,  # height of the figure
    margin = dict(l=0, r=0, t=30, b=0)  # reduce margins on sides
)

fig.show()